<a href="https://colab.research.google.com/github/ludovicogiannetti/Codici-Tirocinio-LLM/blob/main/Training_SD_1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade fsspec
!pip install -q safetensors==0.4.3 datasets==2.20.0 bitsandbytes==0.45.2 wandb==0.17.6
!pip install -q peft==0.12.0 transformers==4.45.2 diffusers==0.29.2 accelerate==0.34.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.0/201.0 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.0.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.10.0 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2025.10.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 12.6

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
from diffusers import StableDiffusionPipeline
from datasets import load_dataset
import torch, wandb, os, random
from torch import nn
from tqdm import tqdm
from PIL import Image
import numpy as np
import torch.nn.functional as F
from matplotlib import pyplot as plt
from torch.amp import autocast, GradScaler


wandb.init(project="sd15_lora_training", name="sd-15-Simpson")
device = "cuda" if torch.cuda.is_available() else "cpu"

print("Device:", device)
print("Torch:", torch.__version__)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Device: cpu
Torch: 2.8.0+cu126


In [4]:
model_id = "sd-legacy/stable-diffusion-v1-5"

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
)
try:
    pipe.enable_attention_slicing()
    print("Attention slicing enabled.")
except Exception as e:
    print("Warning enabling attention slicing:", e)

try:
    pipe.enable_vae_slicing()
    print("VAE slicing enabled.")
except Exception as e:
    print("Warning enabling VAE slicing:", e)

pipe.to("cuda", dtype=torch.float16)
tokenizer = pipe.tokenizer
text_encoder = pipe.text_encoder
vae = pipe.vae
unet = pipe.unet

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Attention slicing enabled.
VAE slicing enabled.


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
# ! RUN PER SUBSET LAION ! #
# !                      ! #

from google.colab import drive
drive.mount('/content/drive')
import os
from datasets import load_from_disk

dataset_path = '/content/drive/MyDrive/Dataset-Lion-10k/laion_subset_10000'

if os.path.exists(dataset_path):
    print("Cartella laion_subset_10000 trovata!")
    print("Contenuto:")
    print(os.listdir(dataset_path))

    try:
        dataset = load_from_disk(dataset_path)
        print("\n Dataset caricato con successo!")
        print(f"Dimensione dataset: {len(dataset)} esempi")
        print(f"Tipo: {type(dataset)}")

        if hasattr(dataset, 'features'):
            print(f"Features: {dataset.features}")


    except Exception as e:
        print(f"Errore nel caricamento: {e}")

else:
    print("Cartella laion_subset_10000 non trovata!")

In [ ]:
# ! RUN PER SUBSET LION ! # # ! ! #
import requests
from io import BytesIO
from torchvision import transforms

preprocess = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5],
     [0.5, 0.5, 0.5])
    ])
def safe_load_image(url):
  try:
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers, timeout=10)
    response.raise_for_status()
    img = Image.open(BytesIO(response.content))
    if img.mode not in ["RGB", "RGBA"]:
      img = img.convert("RGB")
    return preprocess(img)
  except Exception as e:
    return None
urls = dataset['url']
images = []
captions = []
for url, caption in zip(urls, dataset['caption']):
  img_tensor = safe_load_image(url)
  if img_tensor is not None:
    images.append(img_tensor)
    captions.append(caption)
print(f"Immagini valide: {len(images)}")





KeyboardInterrupt: 

In [ ]:
# ! RUN PER DATASET SIMPSON ! #
# !                         ! #

from torchvision import transforms
from datasets import load_dataset

dataset = load_dataset(
    "JerryMo/image-caption-blip-for-training",
    split="train"
)

preprocess = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

images = []
captions = []

for i, example in enumerate(dataset):
    try:
        img = example["image"].convert("RGB")
        tensor = preprocess(img)
        images.append(tensor)
        captions.append(example["text"])
    except Exception as e:
        print(f"Errore al campione {i}: {e}")

    if i >= 10000:
        break

print(f"Immagini valide: {len(images)}")

In [ ]:
from diffusers import DDPMScheduler
from diffusers import UNet2DConditionModel, AutoencoderKL
from peft import LoraConfig
from diffusers.training_utils import EMAModel
from diffusers import StableDiffusionXLPipeline
from diffusers import UNet2DConditionModel
from transformers import AutoTokenizer
from peft import LoraConfig, get_peft_model
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm.auto import tqdm

noise_scheduler = DDPMScheduler.from_pretrained(model_id, subfolder="scheduler")

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
        "to_q", "to_k", "to_v", "to_out.0",  # attention layers
        "proj_in", "proj_out",  # projection layers
        "ff.net.0.proj", "ff.net.2",  # feed-forward layers
        "conv1", "conv2", "conv_shortcut"  # conv layers
    ],
    lora_dropout=0.1,
    bias="none",
)

output_dir = "./checkpoints_sd15_lora"
os.makedirs(output_dir, exist_ok=True)
train_batch_size = 16
num_epochs = 3
checkpoint_steps = 500
learning_rate = 1e-6




In [ ]:
unet_lora = get_peft_model(unet, lora_config)
print("Parametri LoRA addestrabili:", sum(p.numel() for p in unet_lora.parameters() if p.requires_grad))

Parametri LoRA addestrabili: 7584768


In [ ]:
# ! PER SUBSET LION ! #
# !                 ! #

class ImageCaptionDataset(torch.utils.data.Dataset):
    def __init__(self, images, captions, tokenizer):
        self.images = images
        self.captions = captions
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        pixel_values = self.images[idx]
        caption = self.captions[idx]
        tokens = self.tokenizer(
            caption,
            padding="max_length",
            max_length=77,
            truncation=True,
            return_tensors="pt"
        )
        return {
            "pixel_values": pixel_values,
            "input_ids": tokens.input_ids.squeeze(0)
        }

dataset_torch = ImageCaptionDataset(images, captions, tokenizer)
dataloader = DataLoader(dataset_torch, batch_size=train_batch_size, shuffle=True)

In [ ]:
from torch.optim import AdamW
from diffusers.optimization import get_scheduler

optimizer = AdamW([p for p in unet_lora.parameters() if p.requires_grad], lr=learning_rate, eps=1e-6)
scaler = GradScaler("cuda")

lr_scheduler = get_scheduler(
    name="cosine",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=len(dataloader) * num_epochs
)


In [ ]:
pipe.to("cuda", dtype=torch.float16)

unet_lora = get_peft_model(unet, lora_config)
pipe.unet = unet_lora

print("VAE device:", next(pipe.vae.parameters()).device)
print("UNet device:", next(pipe.unet.parameters()).device)
print("Text encoder device:", next(pipe.text_encoder.parameters()).device)

VAE device: cuda:0
UNet device: cuda:0
Text encoder device: cuda:0


In [ ]:
for name, param in unet_lora.named_parameters():
    if "lora" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

In [ ]:
print("Parametri LoRA addestrabili:", sum(p.numel() for p in unet_lora.parameters() if p.requires_grad))

Parametri LoRA addestrabili: 7584768


In [ ]:
def save_checkpoint_and_sample(step_label):
    ckpt_dir = os.path.join(output_dir, f"step_{step_label}")
    os.makedirs(ckpt_dir, exist_ok=True)
    unet_lora.save_pretrained(ckpt_dir)

    prompt = "The simpsons character"
    pipe.safety_checker = lambda images, **kwargs: (images, [False] * len(images))
    with torch.no_grad():
        image = pipe(prompt, num_inference_steps=30, guidance_scale=7.5).images[0]
        image.save(os.path.join(ckpt_dir, "sample.png"))
        wandb.log({f"sample_{step_label}": wandb.Image(image)})
    print(f"Checkpoint + immagine salvati in {ckpt_dir}")

optimizer = AdamW([p for p in unet_lora.parameters() if p.requires_grad], lr=learning_rate, eps=1e-6)


global_step = 0
save_checkpoint_and_sample("base")
unet_lora.train()
unet_lora.enable_gradient_checkpointing()

for epoch in range(num_epochs):
    for step, batch in enumerate(tqdm(dataloader)):
        global_step += 1

        pixel_values = batch["pixel_values"].to(device, dtype=torch.float16)
        input_ids = batch["input_ids"].to(next(text_encoder.parameters()).device)

        optimizer.zero_grad()

        with autocast("cuda", dtype=torch.float16):

            latents = vae.encode(pixel_values).latent_dist.sample() * 0.18215
            noise = torch.randn_like(latents)
            timesteps = torch.randint(0, 1000, (latents.shape[0],), device=latents.device, dtype=torch.long)
            noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

            encoder_hidden_states = text_encoder(input_ids)[0]
            encoder_hidden_states = encoder_hidden_states.to(next(unet_lora.parameters()).device)

            noise_pred = unet_lora(noisy_latents, timesteps, encoder_hidden_states).sample

            loss = F.mse_loss(noise_pred, noise)

            scaler.scale(loss).backward()

        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()

        lr_scheduler.step()

        wandb.log({"loss": loss.item(), "epoch": epoch, "step": global_step})

        if global_step % 100 == 0:
            print(f"Epoch {epoch} | Step {step} | Loss: {loss.item():.4f}")

        if global_step == len(dataloader) // 2:
            save_checkpoint_and_sample("mid")
        if global_step % checkpoint_steps == 0:
            save_checkpoint_and_sample(f"step_{global_step}")